In [13]:
!pip install tensorflow

In [84]:
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [85]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [86]:
dataset = pd.read_csv("/Users/boramert/Desktop/Code/bil570proje/archive/dataset.csv")

In [87]:
labels = dataset['Label']
data = dataset.loc[:, dataset.columns != 'Label']

In [88]:
labels.value_counts()

0    663808
1    193354
2    187589
Name: Label, dtype: int64

In [89]:
labels.loc[labels == 'Benign'].shape

(0,)

In [90]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size = 0.2, random_state = 12345, stratify = labels)

In [91]:
y_train.value_counts()

0    531046
1    154683
2    150071
Name: Label, dtype: int64

In [95]:
benign_train_data = x_train.loc[y_train == 'Benign']
attack_train_data = x_train.loc[y_train != 'Benign']
benign_test_data = x_test.loc[y_test == 'Benign']
attack_test_data = x_test.loc[y_test != 'Benign']

In [98]:
attack_train_data.shape

(835800, 87)

In [70]:
y_train.loc[y_train != 'Benign'] = 1
y_train.loc[y_train == 'Benign'] = 2
y_test.loc[y_test != 'Benign'] = 1
y_test.loc[y_test == 'Benign'] = 2

In [71]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

In [75]:
benign_train_data.shape

(0, 87)

In [80]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(140, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [77]:
autoencoder.compile(optimizer='adam', loss='mae')

In [78]:
history = autoencoder.fit(benign_train_data, benign_train_data, 
          epochs=20, 
          batch_size=512,
          validation_data=(x_test, x_test),
          shuffle=True)

Epoch 1/20


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.